<a href="https://colab.research.google.com/github/MarioSigal/Aprendizaje-Automatico-I-y-II/blob/main/TP2/TP_2_Aprendizaje_Automatico.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Imports


In [1]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from transformers import BertTokenizer, BertModel
from typing import List, Dict, Any
import numpy as np
import pandas as pd

#DATA FRAME


In [7]:
df = pd.read_csv("dataset_datos_total_grup8.csv")

In [20]:
#leer el dataframe


# Agrupar tokens por párrafo
df_por_parrafos = df.groupby("grupo").agg({
    "instancia_id":list,
    "token_id":list,
    "token": list,
    "punt_inicial": list,
    "punt_final": list,
    "capitalización": list
}).reset_index()


In [22]:
print(df_por_parrafos["instancia_id"][1])

[8, 8, 8, 8, 8, 8, 8, 8, 9, 9, 9, 9, 9, 9, 9, 10, 10, 10, 10, 10, 10, 10, 10, 11, 11, 11, 11, 11, 11, 11, 11, 11, 12, 12, 12, 12, 12, 12, 12, 13, 13, 13, 13, 13, 13, 13, 13, 13, 14, 14, 14, 14, 14, 14, 14, 14, 14, 15, 15, 15, 15, 15, 15, 15, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 10, 10, 10, 10, 11, 11, 11, 11, 11, 12, 12, 12, 12, 12, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 14, 14, 14, 14, 14, 14, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 8, 8, 8, 8, 9, 9, 9, 10, 10, 10, 11, 11, 11, 11, 11, 12, 12, 12, 12, 12, 12, 12, 12, 13, 13, 13, 13, 13, 14, 14, 14, 14, 14, 14, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12,

In [16]:
print(len(df_por_parrafos["token"][2]))
print(df.shape)

7372
(10116012, 7)


##TOKENIZER & BERT EMBEDDINGS

In [4]:
tokenizer = BertTokenizer.from_pretrained("bert-base-multilingual-cased")
model = BertModel.from_pretrained("bert-base-multilingual-cased")
model.eval()

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/625 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/714M [00:00<?, ?B/s]

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(119547, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0-11): 12 x BertLayer(
        (attention): BertAttention(
          (self): BertSdpaSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=Fals

In [5]:
def get_multilingual_token_embedding(token: str):
  """
    Devuelve el embedding (estático) para el token.
  """
  token_id = tokenizer.convert_tokens_to_ids(token)
  if token_id is None or token_id == tokenizer.unk_token_id:
    print(f"❌ El token '{token}' no pertenece al vocabulario de multilingual BERT.")
    return None
  embedding_vector = model.embeddings.word_embeddings.weight[token_id]
  print(f"✅ Token: '{token}' | ID: {token_id}")
  print(f"Embedding shape: {embedding_vector.shape}")
  return embedding_vector

In [ ]:
texto = "a qué hora pasa el ciento siete"
tokens = tokenizer.tokenize(texto)
print(tokens)
# ['a', 'qué', 'hora', 'pasa', 'el', 'cien', '##to', 'siete']
tokens_id = tokenizer.convert_tokens_to_ids(tokens)
print(tokens_id)
# [169, 38188, 24301, 26088, 10125, 99485, 10340, 28394]
embedding_vector = model.embeddings.word_embeddings.weight[tokens_id]
print(embedding_vector.shape)
# torch.Size([8, 768]

['a', 'qué', 'hora', 'pasa', 'el', 'cien', '##to', 'siete']
[169, 38188, 24301, 26088, 10125, 99485, 10340, 28394]
torch.Size([8, 768])


In [ ]:
#Precomputo de embeddings
embeddings_list = []

for tokens in df_por_parrafos["token"]:
    token_embeddings = []
    for token in tokens:
        token_id = tokenizer.convert_tokens_to_ids(token)
        if token_id is None or token_id == tokenizer.unk_token_id:
            token_id = tokenizer.unk_token_id
        embedding_vector = model.embeddings.word_embeddings.weight[token_id]
        token_embeddings.append(embedding_vector)
    paragraph_tensor = torch.stack(token_embeddings).detach()  # 🔹 importante detach
    embeddings_list.append(paragraph_tensor)

df_por_parrafos["embeddings"] = embeddings_list



In [ ]:
print(df_por_parrafos)

      instancia_id                                           token_id  \
0                0  [178, 179, 45006, 11945, 10413, 10104, 15761, ...   
1                1  [10228, 24922, 21744, 82405, 11681, 96739, 106...   
2                2  [10110, 10185, 40790, 10109, 38532, 16924, 150...   
3                3  [10228, 12477, 74288, 90034, 11153, 10173, 335...   
4                4  [10182, 74371, 14386, 10104, 31040, 60083, 106...   
...            ...                                                ...   
3278          3278  [54866, 10127, 10212, 96503, 16085, 10795, 649...   
3279          3279  [94354, 16719, 17518, 10406, 54373, 10537, 101...   
3280          3280  [10840, 12343, 10107, 46934, 10157, 10141, 349...   
3281          3281  [46934, 10157, 10152, 75980, 56474, 10443, 101...   
3282          3282  [10220, 32784, 10165, 45006, 11945, 10110, 116...   

                                                  token  \
0     [j, k, row, ##ling, 31, de, julio, de, 1965, y...   
1    

##CREAR CLASE DATASET PARA PYTORCH

In [ ]:
from torch.utils.data import Dataset

class Dataset(Dataset):
    """clase Dataset de pytorch"""
    def __init__(self,df):
        self.df = df

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        embeddings = torch.tensor(np.array(row["embeddings"]),dtype=torch.float)

        labels = {
            "punt_inicial": torch.tensor(row["punt_inicial"]),
            "punt_final": torch.tensor(row["punt_final"]),
            "capitalización": torch.tensor(row["capitalización"])
        }

        return embeddings, labels

##PADDING

In [ ]:
from torch.nn.utils.rnn import pad_sequence

def collate_fn(batch):
    """
    batch: lista de tuplas (embeddings, labels)
    """
    embeddings_list, labels_list = zip(*batch)

    # Pad embeddings (seq_len, embedding_dim) -> (batch_size, max_seq_len, embedding_dim)
    embeddings_padded = pad_sequence(embeddings_list, batch_first=True, padding_value=0.0)

    # Pad labels
    punt_inicial = pad_sequence([l["punt_inicial"] for l in labels_list], batch_first=True, padding_value=-100)
    punt_final = pad_sequence([l["punt_final"] for l in labels_list], batch_first=True, padding_value=-100)
    capitalizacion = pad_sequence([l["capitalización"] for l in labels_list], batch_first=True, padding_value=-100)

    return embeddings_padded, {
        "punt_inicial": punt_inicial,
        "punt_final": punt_final,
        "capitalizacion": capitalizacion
    }

## DATA LOADER

In [ ]:
#TEST
df_por_parrafos_test = df_por_parrafos.head(100)
print(df_por_parrafos_test)

    instancia_id                                           token_id  \
0              0  [178, 179, 45006, 11945, 10413, 10104, 15761, ...   
1              1  [10228, 24922, 21744, 82405, 11681, 96739, 106...   
2              2  [10110, 10185, 40790, 10109, 38532, 16924, 150...   
3              3  [10228, 12477, 74288, 90034, 11153, 10173, 335...   
4              4  [10182, 74371, 14386, 10104, 31040, 60083, 106...   
..           ...                                                ...   
95            95  [10109, 26142, 31285, 10288, 10104, 46934, 101...   
96            96  [36579, 12655, 33198, 10225, 22536, 10109, 623...   
97            97  [13672, 18278, 10157, 12028, 10690, 12655, 332...   
98            98  [10795, 14668, 11504, 61816, 10911, 10228, 380...   
99            99  [18419, 66558, 31355, 46934, 10157, 10192, 115...   

                                                token  \
0   [j, k, row, ##ling, 31, de, julio, de, 1965, y...   
1   [ha, escrito, asi, ##mismo, t

In [ ]:
from torch.utils.data import DataLoader

dataset = Dataset(df_por_parrafos_test)
dataloader = DataLoader(dataset, batch_size=8, shuffle=True, collate_fn=collate_fn)

##RNN Unidireccional

####Encoder

In [ ]:
class EncoderUnidireccional(nn.Module):
    def __init__(self, embedding_dim=768, hidden_dim=256, num_layers=2, dropout=0.3):
        super(EncoderUnidireccional, self).__init__()
        self.lstm = nn.LSTM(
            input_size=embedding_dim,
            hidden_size=hidden_dim,
            num_layers=num_layers,
            batch_first=True,
            dropout=dropout,
            bidirectional=False  # unidireccional
        )

    def forward(self, embeddings):
        """
        embeddings: tensor de forma (batch_size, seq_len, embedding_dim)
        """
        outputs, (hidden, cell) = self.lstm(embeddings)
        # outputs: (batch_size, seq_len, hidden_dim)
        # hidden: (num_layers, batch_size, hidden_dim)
        # cell:   (num_layers, batch_size, hidden_dim)
        return outputs, (hidden, cell)

####Decoder

In [ ]:
class DecoderUnidireccional(nn.Module):
    def __init__(self, hidden_dim=256, num_layers=2, dropout=0.3):
        super(DecoderUnidireccional, self).__init__()
        self.lstm = nn.LSTM(
            input_size=hidden_dim,
            hidden_size=hidden_dim,
            num_layers=num_layers,
            batch_first=True,
            dropout=dropout,
            bidirectional=False  # unidireccional
        )

        # Capa feed-forward para cada problema
        self.punt_inicial_ff = nn.Linear(hidden_dim, 2)
        self.punt_final_ff = nn.Linear(hidden_dim, 4)
        self.capital_ff = nn.Linear(hidden_dim, 4)

        # Función de activación para cada problema
        #self.punt_inicial_sigmoid = nn.Sigmoid()
        #self.punt_final_softmax = nn.Softmax(dim=4)
        #self.capital_softmax = nn.Softmax(dim=4)


    def forward(self, encoder_outputs, hidden, cell):
        """
        encoder_outputs: (batch_size, seq_len, hidden_dim)
        hidden, cell: del encoder
        """
        outputs, _ = self.lstm(encoder_outputs, (hidden, cell))

        #punt_inicial_logits = self.punt_inicial_sigmoid(self.punt_inicial_ff(outputs))
        #punt_final_logits = self.punt_final_sofmax(self.punt_final_ff(outputs))
        #capital_logits = self.capital_sofmax(self.capital_ff(outputs))

        punt_inicial_logits = self.punt_inicial_ff(outputs)
        punt_final_logits = self.punt_final_ff(outputs)
        capital_logits = self.capital_ff(outputs)


        return {
            "puntuación inicial": punt_inicial_logits,
            "puntuación final": punt_final_logits,
            "capitalización": capital_logits,
        }


####Encoder–Decoder

In [ ]:
class ModeloUnidireccional(nn.Module):
    def __init__(self, embedding_dim=768, hidden_dim=256, num_layers=2, dropout=0.3):
        super(ModeloUnidireccional, self).__init__()
        self.encoder = EncoderUnidireccional(embedding_dim, hidden_dim, num_layers, dropout)
        self.decoder = DecoderUnidireccional(hidden_dim, num_layers, dropout)

    def forward(self, embeddings):
        encoder_outputs, (hidden, cell) = self.encoder(embeddings)
        predictions = self.decoder(encoder_outputs, hidden, cell)
        return predictions


## RNN Bidireccional


#### Encoder


In [ ]:
class EncoderBidireccional(nn.Module):
    def __init__(self, embedding_dim=768, hidden_dim=256, num_layers=2, dropout=0.3):
        super(EncoderBidireccional, self).__init__()
        self.lstm = nn.LSTM(
            input_size=embedding_dim,
            hidden_size=hidden_dim,
            num_layers=num_layers,
            batch_first=True,
            dropout=dropout,
            bidirectional=True  # bidireccional
        )

    def forward(self, embeddings):
        """
        embeddings: tensor de forma (batch_size, seq_len, embedding_dim)
        """
        outputs, (hidden, cell) = self.lstm(embeddings)
        # outputs: (batch_size, seq_len, hidden_dim)
        # hidden: (num_layers, batch_size, hidden_dim)
        # cell:   (num_layers, batch_size, hidden_dim)
        return outputs, (hidden, cell)

####Decoder

In [ ]:
class DecoderBidireccional(nn.Module):
    def __init__(self, hidden_dim=256, num_layers=2, dropout=0.3):
        super(DecoderBidireccional, self).__init__()
        self.lstm = nn.LSTM(
            input_size=hidden_dim,
            hidden_size=hidden_dim,
            num_layers=num_layers,
            batch_first=True,
            dropout=dropout,
            bidirectional=True  # bidireccional
        )

        # Capa feed-forward para cada problema
        self.punt_inicial_ff = nn.Linear(hidden_dim, 2)
        self.punt_final_ff = nn.Linear(hidden_dim, 4)
        self.capital_ff = nn.Linear(hidden_dim, 4)

        # Función de activación para cada problema
        #self.punt_inicial_sigmoid = nn.Sigmoid()
        #self.punt_final_softmax = nn.Softmax(dim=4)
        #self.capital_softmax = nn.Softmax(dim=4)


    def forward(self, encoder_outputs, hidden, cell):
        """
        encoder_outputs: (batch_size, seq_len, hidden_dim)
        hidden, cell: del encoder
        """
        outputs, _ = self.lstm(encoder_outputs, (hidden, cell))

        #punt_inicial_logits = self.punt_inicial_sigmoid(self.punt_inicial_ff(outputs))
        #punt_final_logits = self.punt_final_sofmax(self.punt_final_ff(outputs))
        #capital_logits = self.capital_sofmax(self.capital_ff(outputs))

        punt_inicial_logits = self.punt_inicial_ff(outputs)
        punt_final_logits = self.punt_final_ff(outputs)
        capital_logits = self.capital_ff(outputs)

        return {
            "puntuación inicial": punt_inicial_logits,
            "puntuación final": punt_final_logits,
            "capitalización": capital_logits,
        }


####Encoder-Decoder

In [ ]:
class ModeloBidireccional(nn.Module):
    def __init__(self, embedding_dim=768, hidden_dim=256, num_layers=2, dropout=0.3):
        super(ModeloBidireccional, self).__init__()
        self.encoder = EncoderBidireccional(embedding_dim, hidden_dim, num_layers, dropout)
        self.decoder = DecoderBidireccional(hidden_dim, num_layers, dropout)

    def forward(self, embeddings):
        encoder_outputs, (hidden, cell) = self.encoder(embeddings)
        predictions = self.decoder(encoder_outputs, hidden, cell)
        return predictions


##ENTRENAR MODELO


In [ ]:
model = ModeloUnidireccional(embedding_dim=768, hidden_dim=256, num_layers=2)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
criterion = torch.nn.CrossEntropyLoss(ignore_index=-100)  # ignorar padding
num_epochs = 1
for epoch in range(num_epochs):
    model.train()
    for embeddings, labels in dataloader:
        optimizer.zero_grad()

        outputs = model(embeddings)  # diccionario con tus tres salidas

        loss_inicial = criterion(outputs["puntuación inicial"].permute(0,2,1), labels["punt_inicial"])
        loss_final = criterion(outputs["puntuación final"].permute(0,2,1), labels["punt_final"])
        loss_cap = criterion(outputs["capitalización"].permute(0,2,1), labels["capitalizacion"])

        loss = loss_inicial + loss_final + loss_cap
        loss.backward()
        optimizer.step()
